In [ ]:
import scipp as sc
import numpy as np
import matplotlib.pyplot as plt
import plopp as pp
%matplotlib widget

## Load vanadium data
number_of_Q_points=16

#Preallocate
intensity_values=np.zeros((number_of_Q_points,1024))
error_values=np.zeros((number_of_Q_points,1024))

# Load data into a matrix
for Q in range(16):
    filename = '../../IN16b_GGG_data/vanadium_Q' +str(Q+1) +'.dat'

    data_array = np.loadtxt(filename)
    energy_values=data_array[:, 0] #should be the same for all Q
    # EnergyValues[Q,:]=data_array[:, 0]
    intensity_values[Q,:]=data_array[:,1]
    error_values[Q,:]=data_array[:,2]

# Define energy, q and intensity as scipp variables with units, and make a DataArraw
energy=sc.array(dims=['energy'],values=energy_values/1000,unit='meV')
Q=sc.array(dims=['Q'],values=range(number_of_Q_points))
intensity=sc.array(dims=['Q','energy'],values=intensity_values,variances=error_values*error_values) #The variance is the square of the uncertainty!

vanadium_data = sc.DataArray(data=intensity, coords={'Q':Q,'energy': energy})

## Bin and plot data 

Q_bins=16
energy_bins=sc.scalar(1e-3*0.2, unit='meV')

intensity_min=0.0
intensity_max=0.06

energy_min = -0.02 * sc.Unit('meV')
energy_max = 0.02 * sc.Unit('meV')

# da.coords['Energy'] = da.coords['Energy'].to(unit='micro*eV') #optional change the scale to mueV
binned_vanadium_data=vanadium_data.flatten(to='dummy').bin(energy=energy_bins,Q=Q_bins).bins.mean() #can add .plot() to plot it

binned_vanadium_data.plot()



In [ ]:

# Slice the binned data
pp.slicer(binned_vanadium_data['energy',energy_min:energy_max],vmin=intensity_min,vmax=intensity_max,
     keep=['energy'],
     linestyle='none',
     marker='o',
     markerfacecolor='none',
     color='k'
)





In [ ]:
# Fit a single slice of data
from easyCore.Objects.Variable import Parameter
from easyCore.Objects.ObjectClasses import BaseObj
from easyCore.Fitting.Fitting import Fitter


#Assume just a Gaussian for now
#TODO: Would be neat to do this with units
energy_offset_guess=-0.5*1e-3
res_gauss1_area_guess=0.01
res_gauss1_sigma_guess=0.3*1e-3
res_BG_guess=1e-4

energy_offset    =Parameter(name='energy_offset',   value=energy_offset_guess   , fixed=False)
res_gauss1_area  =Parameter(name='res_gauss1_area', value=res_gauss1_area_guess , fixed=False)
res_gauss1_sigma =Parameter(name='res_gauss1_sigma',value=res_gauss1_sigma_guess, fixed=False)
res_BG =Parameter(name='res_BG',value=res_BG_guess,fixed=False)

# TODO: What is up with this way of doing things; the function takes parameters from outside the function???
# TODO: How to use functions from QENSlibrary?
def resolution_function(x: np.ndarray) -> np.ndarray:
    """
    (Gaussian) resolution function
    
    :x: values to calculate the model over. 
    
    :return: model values.
    """

    x=x-energy_offset.raw_value

    y=res_gauss1_area.raw_value*1/np.sqrt(2*np.pi)/res_gauss1_sigma.raw_value*np.exp(-0.5*(x/res_gauss1_sigma.raw_value)**2  )+res_BG.raw_value

    return y

resolution = BaseObj(name='resolution', energy_offset=energy_offset, res_gauss1_area=res_gauss1_area, res_gauss1_sigma=res_gauss1_sigma)
f = Fitter(resolution, resolution_function)


In [ ]:
binned_vanadium_data['energy',energy_min:energy_max]


In [29]:


# energy_to_be_fitted   =        binned_vanadium_data['energy',energy_min:energy_max].bins.mean().coords.get('energy').values
energy_to_be_fitted=   binned_vanadium_data['energy',energy_min:energy_max].coords.get('energy').values
energy_to_be_fitted   = (energy_to_be_fitted[1]-energy_to_be_fitted[0])/2+energy_to_be_fitted[:-1]
intensity_to_be_fitted=binned_vanadium_data['energy',energy_min:energy_max].values
error_to_be_fitted=np.sqrt(binned_vanadium_data['energy',energy_min:energy_max].variances)

for Q_index in [5]:
    fit_result = f.fit(x=energy_to_be_fitted, y=intensity_to_be_fitted[:,Q_index], weights=1/error_to_be_fitted[:,Q_index])
    res_gauss1_area_fit=res_gauss1_area
    res_gauss1_sigma_fit=res_gauss1_sigma


    #TODO: Figure out how to make the x axis of the VanadiumFit different from VanadiumData. Consider of VanadiumData can/should contain everything
    y2=sc.array(dims=['energy'],values=resolution_function(energy_to_be_fitted) )

    VanadiumFit = sc.DataArray(data=y2, coords={'energy': binned_vanadium_data['energy',energy_min:energy_max].coords.get('energy')})
    # sc.plot(VanadiumFit)

    # Plot data and fit
    # Play a bit around with plot options
    IntensityMin=0.0
    IntensityMax=0.06
    sc.plot({'Data': binned_vanadium_data['energy',energy_min:energy_max],
            'Fit': VanadiumFit},
            vmin=IntensityMin,vmax=IntensityMax,marker={'Data':'o','Fit': 'none'},markerfacecolor='none',color={'Data': 'black', 'Fit': 'red'},linestyle={'Data': 'none','Fit': '-'})



ValueError: All items given to the plot function must have the same number of dimensions. Found dimensions {1, 2}.

In [ ]:


energy_to_be_fitted   = (energy_to_be_fitted[1]-energy_to_be_fitted[0])/2+energy_to_be_fitted[:-1]

intensity_to_be_fitted=        binned_vanadium_data['Energy',energy_min:energy_max].bins.mean().values
error_to_be_fitted    =np.sqrt(binned_vanadium_data['Energy',energy_min:energy_max].bins.mean().variances)

fit_result = f.fit(x=energy_to_be_fitted, y=intensity_to_be_fitted, weights=1/error_to_be_fitted)

# Save the resolution parameters, because of the global variable issue

res_gauss1_area_fit=res_gauss1_area
res_gauss1_sigma_fit=res_gauss1_sigma



#TODO: Figure out how to make the x axis of the VanadiumFit different from VanadiumData. Consider of VanadiumData can/should contain everything
y2=sc.array(dims=['Energy'],values=resolution_function(VanadiumData.coords.get('Energy').values)) 

VanadiumFit = sc.DataArray(data=y2, coords={'Energy': Energy})
# sc.plot(VanadiumFit)

 Plot data and fit
# Play a bit around with plot options
IntensityMin=0.0
IntensityMax=0.06
sc.plot({'Data': binnedVanadiumData['Energy',start:stop].bins.mean(),
         'Fit': VanadiumFit},
        vmin=IntensityMin,vmax=IntensityMax,marker={'Data':'o','Fit': 'none'},markerfacecolor='none',color={'Data': 'black', 'Fit': 'red'},linestyle={'Data': 'none','Fit': '-'})

# sc.plot(binnedVanadiumData['Energy',start:stop].bins.mean(),vmin=IntensityMin,vmax=IntensityMax,marker='o',markerfacecolor='none',color='blue',linestyle='--')


# y_fit=resolution_function(energy_to_be_fitted)
# y_fit
# sc.plot(energy_to_be_fitted,y_fit)
